In [1]:
import numpy as np
import scipy as sp
import suite2p as s2p
import TwoPUtils
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
write_folder = os.path.join('/mnt','BigDisk','2P_scratch')


mouse = '4467975.1'

file_list = ({'date': '09_10_2020', 'scene':'TwoTower_foraging','session':4,'scan':11},
             {'date': '11_10_2020', 'scene':'TwoTower_foraging','session':2,'scan':4},
             {'date': '11_10_2020', 'scene':'TwoTower_foraging','session':4,'scan':10},
             {'date': '13_10_2020', 'scene':'TwoTower_foraging','session':2,'scan':5},
             {'date': '14_10_2020', 'scene':'TwoTower_foraging','session':2,'scan':3},
             {'date': '15_10_2020', 'scene':'TwoTower_foraging','session':3,'scan':6},
             {'date': '16_10_2020', 'scene':'TwoTower_foraging','session':3,'scan':7},
             {'date': '17_10_2020', 'scene':'TwoTower_foraging','session':2,'scan':2},
             {'date': '17_10_2020', 'scene':'TwoTower_foraging','session':3,'scan':8}, # dendrites
             {'date': '17_10_2020', 'scene':'TwoTower_foraging','session':7,'scan':14}, # dendrites
             )

In [3]:
def check_and_make_h5s(file_list, runh5=True):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        try:
            source_folder  = os.path.join('/media','mplitt','Backup Plus','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus1','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
        if not os.path.exists(h5path) and runh5:
            _ = TwoPUtils.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=True)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [4]:

for f in file_list:
    
    _h5list,_frs = check_and_make_h5s([f,],runh5=True)
    ops_dir = os.path.join(_h5list[0],'suite2p','plane0','ops.npy')
    s2p_dir = os.path.join(_h5list[0],'suite2p','plane0')
    if os.path.exists(ops_dir):
        ops = np.load(os.path.join(s2p_dir,"ops.npy"),allow_pickle=True).all()
        ops['reg_file'] = os.path.join(s2p_dir,"data.bin")
        ops['reg_file_chan2'] = os.path.join(s2p_dir,"data_chan2.bin")
        np.save(ops_dir,ops)

    # if ops file exists
        # update paths to be local paths
    ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'nchannels':2,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'h5py_key':'data',
                                    'nchannels':2,
                                    'threshold_scaling':1,
                                    'chan2_thresh':.01})
    ops=s2p.run_s2p(ops=ops)
    !rm {_h5list[0]}/*.h5 

{'date': '09_10_2020', 'scene': 'TwoTower_foraging', 'session': 4, 'scan': 11}
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 29091
['/mnt/BigDisk/2P_scratch/4467975.1/09_10_2020/TwoTower_foraging/TwoTower_foraging_004_011']
{}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/mnt/BigDisk/2P_scratch/4467975.1/09_10_2020/TwoTower_foraging/TwoTower_foraging_004_011/TwoTower_foraging_004_011.h5']
time 1824.26 sec. Wrote 29091 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 29091 frames
NOTE: est